# PixelBytes: Catching Insights in Unified Multimodal Sequences

Ce notebook présente **PixelBytes**, un modèle innovant conçu pour générer simultanément du texte et des images pixel par pixel sous forme de séquences. L'objectif est d'explorer un embedding unifié qui permet une génération multimodale cohérente.

## Contexte et Architecture Proposée

### Fondements Théoriques
- **Image Transformer** : [Génération d'images pixel par pixel](https://arxiv.org/abs/1802.05751)
- **Bi-Mamba+** : [Modèle bidirectionnel pour la prévision de séries temporelles](https://arxiv.org/abs/2404.15772)
- **MambaByte** : [Modèle d'état d'espace sélectif sans token](https://arxiv.org/abs/2401.13660)

### Concept Clé
Le modèle PixelByte génère des séquences mixtes de texte et d'images. Il doit :
- Gérer les transitions entre texte et image avec des sauts de ligne (ASCII 0A).
- Maintenir la cohérence des dimensions des images générées.
- Assimiler la tâche de "copie" pour reproduire des motifs complexes.

Ce notebook utilise la puissance des GPU T4 x2 de Kaggle pour expérimenter avec des architectures avancées et des jeux de données volumineux, afin de relever les défis de la génération multimodale unifiée.

## Ressources du Projet

### Dataset
Pour ce projet, nous utiliserons le dataset **PixelBytes-Pokemon**, spécialement conçu pour cette tâche de génération multimodale. Ce dataset, créé par l'auteur de ce notebook, est disponible sur Hugging Face : [PixelBytes-Pokemon](https://huggingface.co/datasets/ffurfaro/PixelBytes-Pokemon). Il contient des séquences de texte et d'images de Pokémon, encodées de manière à permettre l'entraînement de notre modèle PixelByte sur des données multimodales.

### Implémentation
L'implémentation du modèle et les scripts d'entraînement sont disponibles dans le dépôt GitHub **Mamba-Bys** : [Mamba-Bys](https://github.com/fabienfrfr/Mamba-Bys). Ce dépôt contient le code source nécessaire pour reproduire les expériences, ainsi que des instructions détaillées sur la configuration et l'utilisation du modèle PixelByte.

## Modele à entrainer :

- 8 LSTM (bidirectionnel + 1,2,3 layers) + (p_embed + bi-2 layers)
- 6 Mamba (bidirectionnel + 1,2,3 layers)
- 3 Transformers (1,2,3 layers)

# Pre-test

Avant d'entrainer les 8 LSTM, prendre le pembed-bi-2 LSTM et tester la génération (à 40%, mais influence des répétitions de caractére et de pixel)

Forte chance de predire le meme pixel à la suite, et de prédire des espaces et voyelles.

Pour la generation, le modele génére uniquement le prochain element central. L'algorithme de génération doit reconstituer la structure 2D. Tel que : 
(T,T,\n,T,\n,P,P,P,\t,P,P,P,\n,T,T) donne 
([[0,0,0],[0,T,0],[0,0,0]],
[[0,0,0],[T,T,0],[0,0,0]],
[[0,0,0],[T,\n,0],[0,0,0]],
[[0,0,0],[\n,T,0],[0,0,0]],
[[0,0,0],[T,\n,0],[0,0,0]],
[[0,0,0],[0,P,0],[0,0,0]],
[[0,0,0],[P,P,0],[0,0,0]],
[[0,0,0],[P,P,0],[0,0,0]],
[[0,0,0],[P,\t,0],[0,0,0]],
[[0,P,P],[0,P,0],[0,0,0]],
[[P,P,P],[P,P,0],[0,0,0]],
[[P,P,P],[P,P,0],[0,0,0]],
[[P,\t,0],[P,\n,0],[0,0,0]],
[[0,0,0],[0,T,0],[0,0,0]],
[[0,0,0],[T,T,0],[0,0,0]],) avec P les pixel, \n les saut de ligne et de modalité, et \t, les changement de ligne de pixex de l'image et/ou les tabulations de texte.

In [1]:
!pip install -q git+https://github.com/fabienfrfr/PixelBytes.git@main

In [3]:
# only in kaggle for HF
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
# no warning msg during train
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# our approach
from pixelbytes import *

In [4]:
# init
hf_dataset = load_dataset("ffurfaro/PixelBytes-Pokemon")
ds = hf_dataset["train"].train_test_split(test_size=0.1)

# impossible config in Kaggle --> but good way to train apparently
train_dataset = PxByDataset(ds["train"]["pixelbyte"], seq_length=128, stride=1) # 256, 32
test_dataset = PxByDataset(ds["test"]["pixelbyte"], seq_length=128, stride=1)

pixelbyte = PixelBytesTokenizer()
vocab_size = pixelbyte.__len__()

Generating train split:   0%|          | 0/964 [00:00<?, ? examples/s]

In [5]:
### Config LSTM
# modele de reference (LSTM, bidirectionnel, pxby, 81 dim (9 embed), 64 state, 2 layers) (fait)
model_config = ModelConfig(dim=81, d_state=64, depth=2, vocab_size=vocab_size)
# modele LSTM (LSTM, bidirectionnel, center, 81 dim (9 embed), 64 state, 2 layers) #validateur 1 (fait)
model_config = ModelConfig(dim=81, d_state=64, depth=2, vocab_size=vocab_size, pxby_embed=False, pembed=False)
# modele LSTM (LSTM, bidirectionnel, pxby-noconv, 81 dim (9 embed), 64 state, 2 layers) #validateur 2 (fait)
model_config = ModelConfig(dim=81, d_state=64, depth=2, vocab_size=vocab_size, pembed=False)
# modele LSTM (LSTM, unidirectionnel, pxby, 81 dim (9 embed), 64 state, 2 layers) (fait)
model_config = ModelConfig(dim=81, d_state=64, depth=2, vocab_size=vocab_size, bidirectional=False)
# modele LSTM (LSTM, bidirectionnel, pxby, 36 dim (9 embed), 64 state, 2 layers) (fait)
model_config = ModelConfig(dim=36, d_state=64, depth=2, vocab_size=vocab_size)
# modele LSTM (LSTM, bidirectionnel, pxby, 162 dim (18 embed), 64 state, 2 layers) (fait)
model_config = ModelConfig(dim=162, d_state=64, depth=2, vocab_size=vocab_size)
# modele LSTM (LSTM, bidirectionnel, pxby, 81 dim (9 embed), 32 state, 2 layers) (fait)
model_config = ModelConfig(dim=81, d_state=32, depth=2, vocab_size=vocab_size)
# modele LSTM (LSTM, bidirectionnel, pxby, 81 dim (9 embed), 128 state, 2 layers) (fait)
model_config = ModelConfig(dim=81, d_state=128, depth=2, vocab_size=vocab_size)
# modele LSTM (LSTM, bidirectionnel, pxby, 81 dim (9 embed), 64 state, 1 layers) (fait)
model_config = ModelConfig(dim=81, d_state=64, depth=1, vocab_size=vocab_size)
# modele LSTM (LSTM, bidirectionnel, pxby, 81 dim (9 embed), 64 state, 3 layers) (fait)
model_config = ModelConfig(dim=81, d_state=64, depth=3, vocab_size=vocab_size)
# modele LSTM (LSTM, bidirectionnel, pxby, 81 dim (9 embed), 64 state, 2 layers) #special (fait)
model_config = ModelConfig(dim=81, d_state=64, depth=2, vocab_size=vocab_size)

### Model LSTM
model = SimpleRNNModel(model_config)

"""
### Config Mamba
# modele Mamba (Mamba, bidirectionnel, pxby, 81 dim (9 embed), 64 state, 2 layers)
model_config = ModelConfig(dim=81, d_state=64, depth=2, vocab_size=vocab_size)
# modele Mamba (Mamba, unidirectionnel, pxby, 81 dim (9 embed), 64 state, 2 layers)
model_config = ModelConfig(dim=81, d_state=64, depth=2, vocab_size=vocab_size, bidirectional=False)
# modele Mamba (Mamba, bidirectionnel, pxby, 81 dim (9 embed), 64 state, 1 layers)
model_config = ModelConfig(dim=81, d_state=64, depth=1, vocab_size=vocab_size)

### Model Mamba
#model = bMamba(model_config)

### Config Transformer
# modele Transformer (Transformer, //, pxby, 81 dim (9 embed), 64 state, 1 layers)
model_config = ModelConfig(dim=81, d_state=64, depth=1, vocab_size=vocab_size)
# modele Transformer (Transformer, //, pxby, 81 dim (9 embed), 64 state, 2 layers)
model_config = ModelConfig(dim=81, d_state=64, depth=2, vocab_size=vocab_size)

### Model Transformer
model = SimpleTransformerModel(model_config)
"""

## model show
print(model)

SimpleRNNModel(
  (embedding): PxByEmbed(
    (projection): Linear(in_features=81, out_features=81, bias=True)
    (norm): LayerNorm((81,), eps=1e-05, elementwise_affine=True)
    (linear_embedding): Embedding(113, 9)
    (patch_embedding): Conv2d(9, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (_lstm): LSTM(81, 32, batch_first=True, bidirectional=True)
  (lstm): LSTM(64, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=113, bias=True)
)


In [9]:
train_config = TrainConfig(model=model, model_config=model_config, dataset_name="PixelBytes-Pokemon", hf_token=hf_token,
                           train_dataset=train_dataset,test_dataset=test_dataset, num_epochs=1000, repo_name="PixelBytes-Pokemon") # 200
trainer = Trainer(train_config)

Complete path of pytorch model '.pth': models/rnn_bi_pxby_conv_81-dim_64-state_2-layer_PixelBytes-Pokemon


In [ ]:
trainer.train_and_evaluate()

Training:   0%|          | 5/1000 [18:36<62:14:46, 225.21s/it]


Epoch 5: Train Loss: 1.1866, Test Loss: 1.2122, Test Acc: 63.87%


Training:   1%|          | 10/1000 [37:10<61:45:43, 224.59s/it]


Epoch 10: Train Loss: 1.1537, Test Loss: 1.1869, Test Acc: 64.52%


Training:   2%|▏         | 15/1000 [55:44<61:38:13, 225.27s/it]

...

Training:  19%|█▉        | 190/1000 [11:33:38<48:53:56, 217.33s/it]


Epoch 190: Train Loss: 1.0743, Test Loss: 1.1118, Test Acc: 66.41%


Training:  20%|█▉        | 195/1000 [11:51:32<48:28:57, 216.82s/it]


Epoch 195: Train Loss: 1.0742, Test Loss: 1.1093, Test Acc: 66.26%


Training:  20%|█▉        | 196/1000 [11:55:05<48:07:42, 215.50s/it]

In [2]:
ls models

ls: cannot access 'models': No such file or directory


In [12]:
model_ = bMamba.from_pretrained("ffurfaro/PixelBytes-Pokemon", subfolder="ssm_bi_pxby_conv_81_dim_64_state_2_layer_last")
model_

bMamba(
  (embedding): PxByEmbed(
    (projection): Linear(in_features=81, out_features=81, bias=True)
    (norm): LayerNorm((81,), eps=1e-05, elementwise_affine=True)
    (linear_embedding): Embedding(113, 9)
    (patch_embedding): Conv2d(9, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (_mamba): Mamba(
    (in_proj): Linear(in_features=81, out_features=324, bias=False)
    (conv1d): Conv1d(162, 162, kernel_size=(4,), stride=(1,), padding=(3,), groups=162)
    (act): SiLU()
    (x_proj): Linear(in_features=162, out_features=134, bias=False)
    (dt_proj): Linear(in_features=6, out_features=162, bias=True)
    (out_proj): Linear(in_features=162, out_features=81, bias=False)
  )
  (layers): ModuleList(
    (0): Mamba(
      (in_proj): Linear(in_features=81, out_features=324, bias=False)
      (conv1d): Conv1d(162, 162, kernel_size=(4,), stride=(1,), padding=(3,), groups=162)
      (act): SiLU()
      (x_proj): Linear(in_features=162, out_features=134, bias=False)
      (dt